In [2]:
import pandas as pd 
from sqlalchemy import create_engine
import pymysql
import sqlalchemy
import datetime as dt
import numpy as np



In [3]:
airports_csv=pd.read_csv("data/airports.csv",index_col=False)
new_airports=airports_csv.drop(["IATA_CODE","STATE","LATITUDE","LONGITUDE"],axis=1)
airports_csv.head()

,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
0,ABE,Lehigh Valley International Airport,Allentown,PA,USA,40.65236,-75.44040
1,ABI,Abilene Regional Airport,Abilene,TX,USA,32.41132,-99.68190
2,ABQ,Albuquerque International Sunport,Albuquerque,NM,USA,35.04022,-106.60919
3,ABR,Aberdeen Regional Airport,Aberdeen,SD,USA,45.44906,-98.42183
4,ABY,Southwest Georgia Regional Airport,Albany,GA,USA,31.53552,-84.19447


In [4]:
airlines_csv=pd.read_csv("data/airlines.csv",index_col=False)
airlines_csv.head()

,IATA_CODE,AIRLINE
0,UA,United Air Lines Inc.
1,AA,American Airlines Inc.
2,US,US Airways Inc.
3,F9,Frontier Airlines Inc.
4,B6,JetBlue Airways


In [9]:
flights_csv=pd.read_csv("data/flights.csv",dtype={'DEPARTURE_TIME':str,"ARRIVAL_TIME":str})
flights_csv["YEAR"].replace({2015:2020}, inplace=True)
cols=["YEAR","MONTH","DAY"]
flights_csv['DATE'] = flights_csv[cols].apply(lambda x: '-'.join(x.values.astype(str)), axis="columns")
flights_csv['DATE']=pd.to_datetime(flights_csv['DATE'])
flights_csv.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,DATE
0,2020,1,1,4,AS,98,N407AS,ANC,SEA,5,...,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,2020-01-01
1,2020,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,...,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,2020-01-01
2,2020,1,1,4,US,840,N171US,SFO,CLT,20,...,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,2020-01-01
3,2020,1,1,4,AA,258,N3HYAA,LAX,MIA,20,...,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,2020-01-01
4,2020,1,1,4,AS,135,N527AS,SEA,ANC,25,...,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,2020-01-01


In [11]:
new_flight_csv= flights_csv[["DATE","FLIGHT_NUMBER","AIRLINE","TAIL_NUMBER","ORIGIN_AIRPORT","DESTINATION_AIRPORT","DISTANCE","DEPARTURE_TIME","ARRIVAL_TIME"]]
new_flight_csv.dropna(inplace = True)
new_flight_csv.head()

/Users/work/Library/Python/2.7/lib/python/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,DATE,FLIGHT_NUMBER,AIRLINE,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,DISTANCE,DEPARTURE_TIME,ARRIVAL_TIME
0,2020-01-01,98,AS,N407AS,ANC,SEA,1448,2354,0408
1,2020-01-01,2336,AA,N3KUAA,LAX,PBI,2330,0002,0741
2,2020-01-01,840,US,N171US,SFO,CLT,2296,0018,0811
3,2020-01-01,258,AA,N3HYAA,LAX,MIA,2342,0015,0756
4,2020-01-01,135,AS,N527AS,SEA,ANC,1448,0024,0259


In [12]:
print(new_flight_csv.dtypes)


DATE                   datetime64[ns]
FLIGHT_NUMBER                   int64
AIRLINE                        object
TAIL_NUMBER                    object
ORIGIN_AIRPORT                 object
DESTINATION_AIRPORT            object
DISTANCE                        int64
DEPARTURE_TIME                 object
ARRIVAL_TIME                   object
dtype: object


In [13]:
# 
def map1(s):
    if s[:2]=='24':
        return '23' + ':' + s[2:]
    return s[:2] + ':' + s[2:]

new_flight_csv['DEPARTURE'] = new_flight_csv['DEPARTURE_TIME'].apply(map1)
new_flight_csv['DEPARTURE'] = pd.to_datetime(new_flight_csv['DEPARTURE'], format='%H:%M').dt.time



new_flight_csv['ARRIVAL'] = new_flight_csv['ARRIVAL_TIME'].apply(map1)
new_flight_csv['ARRIVAL'] = pd.to_datetime(new_flight_csv['ARRIVAL'], format='%H:%M').dt.time

new_flight_csv.head()

/Users/work/Library/Python/2.7/lib/python/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/Users/work/Library/Python/2.7/lib/python/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/work/Library/Python/2.7/lib/python/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pand

,DATE,FLIGHT_NUMBER,AIRLINE,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,DISTANCE,DEPARTURE_TIME,ARRIVAL_TIME,DEPARTURE,ARRIVAL
0,2020-01-01,98,AS,N407AS,ANC,SEA,1448,2354,0408,23:54:00,04:08:00
1,2020-01-01,2336,AA,N3KUAA,LAX,PBI,2330,0002,0741,00:02:00,07:41:00
2,2020-01-01,840,US,N171US,SFO,CLT,2296,0018,0811,00:18:00,08:11:00
3,2020-01-01,258,AA,N3HYAA,LAX,MIA,2342,0015,0756,00:15:00,07:56:00
4,2020-01-01,135,AS,N527AS,SEA,ANC,1448,0024,0259,00:24:00,02:59:00


In [14]:
airlines_csv = airlines_csv.rename(columns={"IATA_CODE":"AIRLINE","AIRLINE":"AIRLINE_name"})
airlines_csv

,AIRLINE,AIRLINE_name
0,UA,United Air Lines Inc.
1,AA,American Airlines Inc.
2,US,US Airways Inc.
3,F9,Frontier Airlines Inc.
4,B6,JetBlue Airways
5,OO,Skywest Airlines Inc.
6,AS,Alaska Airlines Inc.
7,NK,Spirit Air Lines
8,WN,Southwest Airlines Co.
9,DL,Delta Air Lines Inc.


In [22]:
merged_data= pd.merge(new_flight_csv,airlines_csv,on="AIRLINE",how="inner")
final_csv=merged_data.drop(["AIRLINE","DEPARTURE_TIME","ARRIVAL_TIME"],axis=1)
a=final_csv.drop_duplicates()
flight_airport=a.sample(n=50)
flight_airport.head()

,DATE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,DISTANCE,DEPARTURE,ARRIVAL,AIRLINE_name
4413813,2020-03-26,4039,N8656B,MCI,LAX,1363,17:01:00,18:05:00,Southwest Airlines Co.
3392968,2020-10-26,5589,N150SY,13930,10529,783,11:23:00,14:22:00,Skywest Airlines Inc.
4835192,2020-07-23,248,N7714B,MDW,OAK,1844,19:24:00,22:02:00,Southwest Airlines Co.
4032615,2020-12-18,2777,N626AE,ACT,DFW,89,13:37:00,14:21:00,Atlantic Southeast Airlines
3720212,2020-05-23,3831,N12563,CAK,ORD,343,06:34:00,07:01:00,Atlantic Southeast Airlines


In [23]:
flight_airport=pd.merge(airports_csv,flight_airport,left_on="IATA_CODE",right_on="DESTINATION_AIRPORT")
flight_airport = flight_airport.rename(columns={"CITY":"ARRIVAL_CITY"}).drop(["DESTINATION_AIRPORT","STATE","COUNTRY","LATITUDE","LONGITUDE","AIRPORT"],axis=1)
flight_airport=pd.merge(airports_csv,flight_airport,left_on="IATA_CODE",right_on="ORIGIN_AIRPORT")


In [ ]:
flight_airport = flight_airport.rename(columns={"CITY":"DEPARTURE_CITY"}).drop(["IATA_CODE_x","IATA_CODE_y","LATITUDE","LONGITUDE","STATE","COUNTRY","ORIGIN_AIRPORT"],axis=1)


In [29]:
flight_airport= flight_airport.rename(columns={"AIRPORT":"airport","DEPARTURE_CITY":"departure_city","ARRIVAL_CITY":"arrival_city","DATE":"date","FLIGHT_NUMBER":"flight_number","TAIL_NUMBER":"tail_number","DISTANCE":"distance","DEPARTURE":"departure_time","ARRIVAL":"arrival","AIRLINE_name":"airline_name"})


In [30]:
flight_airport.head()

,airport,departure_city,arrival_city,date,flight_number,tail_number,distance,departure_time,arrival,airline_name
0,Albuquerque International Sunport,Albuquerque,Dallas,2020-05-30,1008,N387SW,580,16:57:00,19:42:00,Southwest Airlines Co.
1,Waco Regional Airport,Waco,Dallas-Fort Worth,2020-12-18,2777,N626AE,89,13:37:00,14:21:00,Atlantic Southeast Airlines
2,Kodiak Airport,Kodiak,Anchorage,2020-05-01,48,N797AS,253,08:03:00,09:00:00,Alaska Airlines Inc.
3,Hartsfield-Jackson Atlanta International Airport,Atlanta,Augusta,2020-04-27,5274,N842AS,143,22:43:00,23:26:00,Atlantic Southeast Airlines
4,Hartsfield-Jackson Atlanta International Airport,Atlanta,Ft. Lauderdale,2020-09-02,1726,N519US,581,13:57:00,15:55:00,Delta Air Lines Inc.


In [31]:
hostname="localhost"
dbname="flight-api"
uname="root"
pwd="haifa3012"

In [32]:
engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}"
				.format(host=hostname, db=dbname, user=uname, pw=pwd))

# Convert dataframe to sql table                                   
flight_airport.to_sql('flight', engine, index=False,if_exists='replace',
            dtype={"date":sqlalchemy.Date(),'flight_number': sqlalchemy.INTEGER(), "tail_number":sqlalchemy.types.VARCHAR(length=50),"departure_city":sqlalchemy.types.VARCHAR(length=50),"arrival_city":sqlalchemy.types.VARCHAR(length=50),
            "distance":sqlalchemy.INTEGER(),"departure_time":sqlalchemy.Time(),"arrival":sqlalchemy.Time(),"airline_name":sqlalchemy.types.VARCHAR(length=255),"airport":sqlalchemy.types.VARCHAR(length=255)})
